# Demonstration of  experimental data fitting using NN model

### Run the cell below to import the libraries

In [ ]:
import sys; sys.path.insert(1, 'code')
from spect_tools import (create_optimizer, CalculatedSpectrum, AtomicElement, uploader, load_file_data, fit_loss)
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

### Run this cell fo load the neural network model

In [ ]:
# Load Deep Neural network for Ti 2p XPS
TRAINED_MODEL_PATH = './trained_models/Ti2pXPS.h5'
trained_model = load_model(TRAINED_MODEL_PATH)

# Energy range in eV for the output of the Ti2pXPS model 
ENERGY_AXIS = np.linspace(448,470,2200) 

# 1 - Load experimental data

### Run this cell and click "Upload" to select your experimental CSV file

In [ ]:
display(uploader)

### Run this cell to load the file data

In [ ]:
CSV_COLUMN_SEPARATOR = '\t' # choose '\t' for tab or ' ' space or ';'
y_exp, energy = load_file_data(CSV_COLUMN_SEPARATOR, ENERGY_AXIS)
plt.plot(energy, y_exp)

# 2 - Fit data

###  Type the initial guess and parameter ranges and run this cell

In [ ]:
# INITIAL GUESS FOR OPTIMIZATION PARAMETERS
ATOMIC_POSITIONS = 2
atom1 = AtomicElement(nox=2, delta=2, Udd=3, Upd=0.5, T2q=-0.5, Dt=0.5, Ds=0)
atom2 = AtomicElement(nox=4, delta=2, Udd=3, Upd=0.5, T2q=4,    Dt=0.5, Ds=0)
atoms = [atom1, atom2]

# OPTIMIZER Settings
S = dict() 
# Instrumental fitting properties
S['range_scale'] = [0.01, 4.0]
S['range_broad'] = [0.05, 2.0] 
S['range_offset']= [-1, 1]
S['range_shift'] = [-3, 3] # energy shift (eV)

# Electronic charge transfer properties
S['range_nox']   = [1.9, 4.1]
S['range_delta'] = [1.0, 2.0]
S['range_Udd'] =   [1.0, 3.0]
S['range_Upd'] =   [0.0, 1.0]

# Cristal distortion
S['range_T2q'] = [-0.6, 4.2]
S['range_Dt'] =  [0.0, 1.1]
S['range_Ds'] =  [0.0, 0.1]

# Optimization parameters
S['set_ftol_rel'] = 1e-6
S['set_xtol_rel'] = 1e-3



### Run this cell to create the optimizer

In [ ]:
# Create fitting optimizer
x_optimum = None
guessed_spectrum = CalculatedSpectrum(trained_model, [], ENERGY_AXIS, ATOMIC_POSITIONS)
guessed_spectrum.broad=1.0
guessed_spectrum.atoms = atoms
initial_parameters = guessed_spectrum.get_parameter_array()

def loss(x, grad):
    return fit_loss(x, y_exp, trained_model, ENERGY_AXIS, ATOMIC_POSITIONS)

fit_optimizer, x0 = create_optimizer(S, loss, num_atoms=ATOMIC_POSITIONS)
print("Optimizer created. Ready to start the fitting")

###  Run this cell to start the fitting ( you can run multiple times)

In [ ]:
# START FITTING
if x_optimum is None: 
    # Run first time 
    x_optimum = fit_optimizer.optimize(initial_parameters)
    
# Run more 10 times    
for i in range(0, 10):  x_optimum = fit_optimizer.optimize(x_optimum)
    
minf = fit_optimizer.last_optimum_value()
print('total error:', minf)

# 3 - Visualize Results

###  Run this cell to visualize the result

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
y_result_spectrum = CalculatedSpectrum(trained_model, x_optimum, ENERGY_AXIS, ATOMIC_POSITIONS)
y_result_spectrum.calculate_spectra(y_exp)
y_result_spectrum.plot(plt,y_exp)
print(y_result_spectrum)
